## Check GPU

In [1]:
!nvidia-smi

Thu Jan  1 18:27:24 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.195.03             Driver Version: 570.195.03     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5070 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   26C    P8             19W /  300W |     633MiB /  16303MiB |      8%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/home/faruk/Documents/nvidia-da/poker-card


## Install YOLO11 via Ultralytics

In [4]:
%pip install "ultralytics<=8.3.40" supervision roboflow wandb

import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 15847MiB)
Setup complete ✅ (16 CPUs, 46.9 GB RAM, 399.5/915.3 GB disk)


## Optional Settings

In [ ]:
import wandb

# Disable ultralytics tracking
!yolo settings sync=False

# Enable W&B logging in Ultralytics  #NEW
!yolo settings wandb=True

In [ ]:
import wandb
wandb.login()

## Fine-tune YOLO11 on custom dataset

**NOTE:** When training YOLOv11, make sure your data is located in `datasets`. If you'd like to change the default location of the data you want to use for fine-tuning, you can do so through Ultralytics' `settings.json`. In this tutorial, we will use one of the [datasets](https://universe.roboflow.com/ks-fsm9o/pelvis-ap-x-ray) available on [Roboflow Universe](https://universe.roboflow.com/). When downloading, make sure to select the `yolov11` export format.

 - Poker Cards Dataset: https://universe.roboflow.com/roboflow-jvuqo/poker-cards-fmjio/dataset/4

In [6]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

/home/faruk/Documents/nvidia-da/poker-card/datasets


In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key=MY_API_KEY)
project = rf.workspace("roboflow-jvuqo").project("poker-cards-fmjio")
version = project.version(4)
dataset = version.download("yolov11")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to poker-cards-4 in yolov11:: 100%|██████████| 1810/1810 [00:00<00:00, 12043.94it/s]


## Custom Training

In [12]:
%cd {HOME}

/home/faruk/Documents/nvidia-da/poker-card


In [13]:

!yolo task=detect mode=train model=yolo11s.pt data={dataset.location}/data.yaml epochs=120 imgsz=640 plots=True project=poker-cards name=run1

100%|██████████████████████████████████████| 18.4M/18.4M [00:01<00:00, 13.4MB/s]
New https://pypi.org/project/ultralytics/8.3.245 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.40 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 15847MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/home/faruk/Documents/nvidia-da/poker-card/datasets/poker-cards-4/data.yaml, epochs=120, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=poker-cards, name=run1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vi

## Thanks 

Original notebook from: https://github.com/roboflow/notebooks